In [2]:
from pathlib import Path
import pyrosetta
import pyrosetta_help as ph
from types import ModuleType
from IPython.display import display, HTML

from collections import Counter
prc: ModuleType = pyrosetta.rosetta.core
prp: ModuleType = pyrosetta.rosetta.protocols
prcc: ModuleType = pyrosetta.rosetta.core.conformation
pr_scoring: ModuleType = pyrosetta.rosetta.core.scoring



ignore_unrecognized_res=False
load_PDB_components=False

logger = ph.configure_logger()
pyrosetta.distributed.maybe_init(extra_options=ph.make_option_string(no_optH=False,
                                                                     ex1=None,
                                                                     ex2=None,
                                                                     # mute='all',
                                                                     ignore_unrecognized_res=ignore_unrecognized_res,
                                                                     load_PDB_components=load_PDB_components,
                                                                     ignore_waters=True)
                                 )
pyrosetta.rosetta.basic.options.set_boolean_option('run:ignore_zero_occupancy', False)
pyrosetta.rosetta.basic.options.set_boolean_option('in:auto_setup_metals', True)

In [6]:
pose = ph.parameterized_pose_from_file(pdb_filename='x0310_raw.pdb', wanted_ligands={'LIG': 'Cc1nn(C)c(C)c1NC(=O)CC#N'})

In [8]:
def relax(original: pyrosetta.Pose,
          constraint_weight: float=5,
          cycles: int=15,
          relax_to_start_coords:bool=True) -> pyrosetta.Pose:
    pose: pyrosetta.Pose = original.clone()
    # configure constraints
    scorefxn: pr_scoring.ScoreFunction = pyrosetta.get_fa_scorefxn()
    scorefxn.set_weight(pr_scoring.ScoreType.coordinate_constraint, constraint_weight)
    scorefxn.set_weight(pr_scoring.ScoreType.angle_constraint, constraint_weight)
    scorefxn.set_weight(pr_scoring.ScoreType.atom_pair_constraint, constraint_weight)
    pyrosetta.rosetta.basic.options.set_boolean_option('relax:constrain_relax_to_start_coords', relax_to_start_coords)
    pyrosetta.rosetta.basic.options.set_boolean_option('relax:coord_constrain_sidechains', relax_to_start_coords)
    # set up the relax sampler
    pyrosetta.rosetta.protocols.relax.FastRelax.register_options()
    relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, cycles)
    relax.constrain_relax_to_start_coords(relax_to_start_coords)
    relax.apply(pose)
    return pose

relax(pose)

pose.dump_pdb('x0310_relaxed.pdb')

True